In [1]:
import numpy as np
import pandas as pd

# text processing libraries
import re
import string
import nltk
nltk.download('stopwords') # bajar la primera vez
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordTokenizer

# sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix,balanced_accuracy_score,roc_curve, auc
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

#lime
!pip install lime # instalar la primera vez
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline



#Lemmatizer
nltk.download('wordnet') # bajar la primera vez
from nltk.stem.wordnet import WordNetLemmatizer

#ngrams
from nltk.util import ngrams
import collections

#spacy
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auror\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
ERROR: Invalid requirement: '#'
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\auror\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ---------------------------------------- 12.8/12.8 MB 2.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


FUNCTIONS

In [2]:
#Limpieza
manual_clean=[] #lista de registros con errores producidos al hacer limpieza
def clean_text(i,text):
 try:
    punctuation_list="""!#$%&'"()*+,-./:;<=>?@[\]^_`{|}~"""
    text = text.strip()
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) #remove text in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) #remove urls, https
    text = re.sub('[%s]' % re.escape(punctuation_list), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) #get rid of digits, ie numbers
    
    #realizando de nuevo la limpieza 
    string=re.sub(r'^(\w)\1{1,}', r'\1', text)#letras que se repiten >=2 veces en la 1era palabra
    string=re.sub(r'(\w)\1{2,}', r'\1', string)#letras que se repiten más de 2 veces
    string=string.strip()
    #letras que se repiten dos o más veces en la palabra final
    final_word=re.findall(r'(\w+$)', string)
    final_word=final_word[0]
    final_new_word=re.sub(r'\b([a-z]{1})\1', r'\1', final_word)
    text=re.sub(r"\w+$", final_new_word, string)#se encarga de las letras que se repiten en el resto de la palabra
    #letras que se repiten dos veces al inicio de cada palabra
    text=re.sub(r'\b([a-z]{1})\1', r'\1', text)
    
 except Exception as error:
    manual_clean.append(i) #i es el número de fila, a estos registros les hare limpieza manual
 else: 
  return text

#este es un ejemplo de lo que hace en la última parte la función anterior
string = "aalejandra aaaaamazzzze in my aabzi wooorld coffee aaabzi abzi well"
string = string.strip()
string=re.sub(r'^(\w)\1{1,}', r'\1', string)
print(string)
string=re.sub(r'(\w)\1{2,}', r'\1', string)
final_word=re.findall(r'(\w+$)', string)
final_word=final_word[0]
print("final word", final_word)
final_new_word=re.sub(r'\b([a-z]{1})\1', r'\1', final_word)
print("final new word", final_new_word)
saludo=re.sub(r"\w+$", final_new_word, string)
print(saludo)
string=re.sub(r'\b([a-z]{1})\1', r'\1', string)
print(string)

alejandra aaaaamazzzze in my aabzi wooorld coffee aaabzi abzi well
final word well
final new word well
alejandra amaze in my aabzi world coffee abzi abzi well
alejandra amaze in my abzi world coffee abzi abzi well


In [3]:
df = pd.read_csv("restaurant_ratings.csv", sep=",")
print(df.columns)
print(df.shape)

Index(['Unnamed: 0', 'name', 'rating', 'review', 'bad'], dtype='object')
(121814, 5)


In [4]:
#eliminamos las columnas name y rating
df.drop(["name","rating"], axis=1, inplace=True)
df.rename(columns={"Unnamed: 0":"id"}, inplace=True)



In [5]:
print(df.shape)
print(df.head(15))


(121814, 3)
         id                                             review  bad
0    228122  Ambience  35  Too noisy\nFood  3\nService  4\n...  0.0
1    400236  If I could  I would rate it in negativeWorst s...  1.0
2     35438                                    Hemanth reddyK   0.0
3    262748  Try out the filter coffee masala dosa idly and...  0.0
4   1008738  Decent Mutton Biryani Consistent I have tried ...  0.0
5    422366  Great ambience good food and best service by t...  0.0
6    438899  Ambiance  The live music and the quirky decor ...  0.0
7    882757  Food tastes really good Very Different Chinese...  0.0
8    868560  Extremely happy about my orderthe food was als...  0.0
9    614196  didnt get half my order  zomato credited me th...  0.0
10  1019204  Ive never been to this place But I have ordere...  0.0
11   132396  Update  They now accept cards\n\nFood review \...  0.0
12   308568  Soya chaap brings back the memories of old Nor...  0.0
13   615715  Have bookmarked this pl

In [6]:
print(df.id.duplicated().sum())#no hay duplicados
#Analizando el número de nulos
df.isnull().sum() #en review tenemos 322 nulos


0


id          0
review    322
bad         0
dtype: int64

In [7]:
df[df.review.isnull()].bad.value_counts() #la mayoría de los nulos son del grupo mayoritario de buenas reseñas

0.0    293
1.0     29
Name: bad, dtype: int64

In [8]:
df.bad.value_counts() #la mayoría son cero de los que son nulos, los eliminaremos estos
df.dropna(inplace=True)

In [9]:
df.reset_index(inplace=True)

In [10]:
df.drop(columns="index",inplace=True)

In [11]:
print(df.shape)
#inicio de limpieza
df_cleaned=df.copy()




(121492, 3)


In [12]:
df_cleaned.loc[:,"review_cleaned"]=""

In [13]:
#aplicando la función de limpieza
rows=df.shape[0]
for i in range(0,rows):
 df_cleaned.review_cleaned.iloc[i]=clean_text(i,df.review.iloc[i])


In [14]:
#observando los registros que proporcionaron error
print("registros con errores ",len(manual_clean))
print(df.bad.iloc[manual_clean].value_counts()) #la mayoria de errores son de buenas reseñas
for i in manual_clean:
 print(df.review.iloc[i],"\n")
#luego de revisarlas, se procederá a borrarlas
df_cleaned.review_cleaned.drop(manual_clean,axis=0,inplace=True)



registros con errores  65
0.0    60
1.0     5
Name: bad, dtype: int64
 

 

       

6  

  

 1MG5   

                        

  9 

 2DJ 

gr8 

 1MG5   

httpmercurialpurpleblogspotin201406notgoodenoughalibihtml 

gr8 

        

       

5 

 

 

    

1 

       

  

  

 

httpmercurialpurpleblogspotin201406notgoodenoughalibihtml 

 

               

                                                       

                                                                                                                                                                                                                    
                                               

 2DJ 

2DJ 

   

MG5BGM 

1MG5 

2DJ 

5 

  

2 

47080BGM 

  

  

 

1MG5 

  

 1MG5   

    

  

2DJ 

  

10 

  

   

  

   

                                   

               

   

   

4 

  

gr8 

10 

  

10111000 

2DJ 

3 

35 



In [15]:
df_cleaned.reset_index(inplace=True)

In [16]:
df_cleaned.drop(columns="index", inplace=True)
df_cleaned

,id,review,bad,review_cleaned
0,228122,Ambience 35 Too noisy\nFood 3\nService 4\n...,0.0,ambience too noisyfood the zomato revie...
1,400236,If I could I would rate it in negativeWorst s...,1.0,if i could i would rate it in negativeworst s...
2,35438,Hemanth reddyK,0.0,hemanth reddyk
3,262748,Try out the filter coffee masala dosa idly and...,0.0,try out the filter coffee masala dosa idly and...
4,1008738,Decent Mutton Biryani Consistent I have tried ...,0.0,decent mutton biryani consistent i have tried ...
...,...,...,...,...
121487,33166,The prices are high for the Quantity and quali...,1.0,the prices are high for the quantity and quali...
121488,642570,Pasta Street has come up with yet another outl...,0.0,pasta street has come up with yet another outl...
121489,480976,Want to give it 35\nNice and quiet ambience\nF...,0.0,want to give it and quiet ambiencefood was go...
121490,34376,will never order from this restaurant I should...,1.0,will never order from this restaurant i should...


In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [18]:
 df_cleaned["review_cleaned_1"]=""


In [19]:
#limitando a ciertas palabras mediante su etiquetado gramatical, el vocabulario que no 
#sean pronombres, simbolos, entre otros...
for i,n in enumerate(df_cleaned.review_cleaned.tolist()):

 doc= nlp(str(n))
 text=[]
 for token in doc:
    if(not token.pos_ in ["AUX","ADP","PROPN","SYM","NUM", "PUNCT"]):
     text.append(token.text)
 df_cleaned["review_cleaned_1"].iloc[i]=" ".join(text)

In [20]:
df_cleaned["tokens_cleaned"]=""
df_cleaned["lemma"]=""

In [21]:
#lista de stopwords
eng_stopwords=stopwords.words("english")
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
nlp = spacy.load('en_core_web_sm')
lemmatizer = nlp.get_pipe("lemmatizer")

#Toquenizaciòn,stopword,lemma
for i,text in enumerate(df_cleaned.loc[:,"review_cleaned_1"].tolist()):
  tokens_final=[]
  tokens_init=tokenizer.tokenize(text)
  tokens_final=[token for token in tokens_init if not token in eng_stopwords] #stopwords
  df_cleaned["tokens_cleaned"].iloc[i]=" ".join(tokens_final)
  doc = nlp(df_cleaned["tokens_cleaned"].iloc[i])
  df_cleaned["lemma"].iloc[i]=" ".join([token.lemma_ for token in doc])








In [22]:

df_cleaned[["tokens_cleaned","lemma"]]

,tokens_cleaned,lemma
0,ambience noisyfood zomato review thought visit...,ambience noisyfood zomato review think visit c...
1,rate shawarma ever diced onion spoils entire t...,rate shawarma ever diced onion spoil entire ta...
2,,
3,try filter coffee masala dosa idly everything ...,try filter coffee masala dosa idly everything ...
4,decent consistent tried times saalan badly mad...,decent consistent try times saalan badly madei...
...,...,...
121487,prices high quantity quality food unhygienic p...,price high quantity quality food unhygienic pl...
121488,come yet another outlet time koramangala conve...,come yet another outlet time koramangala conve...
121489,want give good loved soup disappointed dessert...,want give good love soup disappoint dessert or...
121490,never order restaurant brought icecreams inste...,never order restaurant bring icecream instead ...


In [23]:
#tf-idf
tfdif= TfidfVectorizer(max_features=1500,
                        min_df=2,
                        max_df=0.5,
                        ngram_range=(1, 2))
train_tfidf = tfdif.fit_transform(df_cleaned.lemma)
train_tfidf.toarray()

array([[0.       , 0.       , 0.       , ..., 0.       , 0.3161062,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [24]:
tfidf_df=pd.DataFrame(train_tfidf.toarray(),
             columns=tfdif.get_feature_names_out())

In [25]:
tfidf_df.columns

Index(['able', 'absolute', 'absolutely', 'absolutely love', 'accept',
       'accommodate', 'accompany', 'accord', 'actually', 'add',
       ...
       'year', 'yes', 'yesterday', 'yet', 'young', 'yum', 'yumm', 'yummy',
       'zomato', 'zomato gold'],
      dtype='object', length=1500)

In [26]:
#dividiendo los datos de entrenamiento y test
x=train_tfidf
y=df_cleaned.bad
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1234)

In [27]:
#fitting y métricas del modelo Naive Bayes
model = LogisticRegression()
model = model.fit(x_train, y_train)
y_pred_logistic = model.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test,y_pred_logistic)))
print("Precision: "+str(precision_score(y_test,y_pred_logistic)))
print("Recall: "+str(recall_score(y_test,y_pred_logistic)))
print("F1: "+str(f1_score(y_test,y_pred_logistic)))

Accuracy: 0.9242756804214223
Precision: 0.8452197378565921
Recall: 0.6914222642699464
F1: 0.7606244579358195
